In [831]:
import pandas as pd
import json
import numpy as np

In [832]:
with open("trial_task.json", "r") as f:
    data = f.read()
    json_data = json.loads(data)

In [833]:
orders_data = pd.read_json("trial_task.json") 
orders_data.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [834]:
columns = {'product': [], 'price': [], 'quantity': [], 'order_id': []}
all_products = pd.DataFrame(data=columns)
for line in json_data:
    example = line["products"]
    tmp = pd.DataFrame(data=example)
    tmp['order_id'] = [line["order_id"] for _ in range(np.shape(tmp)[0])]
    all_products = pd.concat([all_products, tmp], ignore_index=True)
all_products['order_id'] = all_products['order_id'].astype(int)
all_products['quantity'] = all_products['quantity'].astype(int)
all_products['price'] = all_products['price'].astype(int)
all_products.head()

,product,price,quantity,order_id
0,ломтик июльского неба,450,1,11973
1,билет в Израиль,1000,3,11973
2,статуэтка Ленина,200,3,11973
3,билет в Израиль,1000,1,62239
4,зеленая пластинка,10,2,85794


In [835]:
orders_data = orders_data.set_index('order_id').join(all_products.set_index('order_id'), on='order_id')\
                            .drop(['products'], axis=1)
orders_data['total_cost_for_each_product'] = orders_data['price'] * orders_data['quantity']
orders_data = orders_data.reset_index(inplace=False)
orders_data.head()

,order_id,warehouse_name,highway_cost,product,price,quantity,total_cost_for_each_product
0,11973,Мордор,-70,ломтик июльского неба,450,1,450
1,11973,Мордор,-70,билет в Израиль,1000,3,3000
2,11973,Мордор,-70,статуэтка Ленина,200,3,600
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1,1000
4,85794,отель Лето,-50,зеленая пластинка,10,2,20


##### 1. Найти тариф стоимости доставки для каждого склада.

In [837]:
warehouses = orders_data['warehouse_name'].unique()
print(warehouses)

['Мордор' 'хутор близ Диканьки' 'отель Лето' 'остров невезения'
 'гиперборея']


In [838]:
d1 = {'warehouse_name' : warehouses, 'shipping_cost' : np.zeros(len(warehouses))}
shipping_cost_rate = pd.DataFrame(data=d1)
shipping_cost_rate

,warehouse_name,shipping_cost
0,Мордор,0.0
1,хутор близ Диканьки,0.0
2,отель Лето,0.0
3,остров невезения,0.0
4,гиперборея,0.0


In [839]:
for i in range(len(warehouses)):
    sum_cost = orders_data[orders_data['warehouse_name'] == warehouses[i]].groupby('order_id', as_index=False)\
                    .aggregate({'highway_cost' : 'mean', 'quantity' : 'sum'})\
                        .sum().drop(['order_id'])
    shipping_cost_rate.at[i, 'shipping_cost'] = sum_cost['highway_cost'] / sum_cost['quantity']

In [841]:
shipping_cost_rate  #тариф стоимости доставки для каждого склада

,warehouse_name,shipping_cost
0,Мордор,-10.0
1,хутор близ Диканьки,-15.0
2,отель Лето,-25.0
3,остров невезения,-5.0
4,гиперборея,-20.0


##### 2. Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit').

In [842]:
orders_data1 = orders_data.join(shipping_cost_rate.set_index('warehouse_name'), on='warehouse_name')
orders_data1['shipping_cost_for_each_product'] = orders_data1['shipping_cost'] * orders_data1['quantity']
orders_data1 = orders_data1.drop(['highway_cost', 'shipping_cost'], axis=1)
orders_data1.head()

,order_id,warehouse_name,product,price,quantity,total_cost_for_each_product,shipping_cost_for_each_product
0,11973,Мордор,ломтик июльского неба,450,1,450,-10.0
1,11973,Мордор,билет в Израиль,1000,3,3000,-30.0
2,11973,Мордор,статуэтка Ленина,200,3,600,-30.0
3,62239,хутор близ Диканьки,билет в Израиль,1000,1,1000,-15.0
4,85794,отель Лето,зеленая пластинка,10,2,20,-50.0


In [843]:
products_information = orders_data1.groupby('product', as_index=False)\
            .aggregate({'quantity' : 'sum', 'total_cost_for_each_product' : 'sum'\
                        , 'shipping_cost_for_each_product' : 'sum'})

products_information = products_information.rename({'total_cost_for_each_product' : 'income'\
                            , 'shipping_cost_for_each_product' : 'expenses'}, axis=1)

products_information['profit'] = (products_information['income'] + products_information['expenses']).astype(int)
products_information

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,-820.0,27980
1,билет в Израиль,58,58000,-1050.0,56950
2,зеленая пластинка,61,610,-920.0,-310
3,ломтик июльского неба,42,18900,-670.0,18230
4,плюмбус,65,16250,-940.0,15310
5,подписка на suppi-блог,33,4950,-545.0,4405
6,статуэтка Ленина,68,13600,-935.0,12665


##### 3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов.

In [844]:
order_id_profit = orders_data.groupby('order_id', as_index=False)\
            .aggregate({'highway_cost' : 'mean','total_cost_for_each_product' : 'sum'})

order_id_profit['order_profit'] = (order_id_profit['total_cost_for_each_product']\
                                             + order_id_profit['highway_cost']).astype(int)

order_id_profit = order_id_profit.drop(['highway_cost', 'total_cost_for_each_product'], axis=1)
order_id_profit.head(10)

,order_id,order_profit
0,124,705
1,1391,490
2,2091,1300
3,2108,200
4,2558,355
5,2751,-40
6,2930,720
7,4035,4125
8,5430,1120
9,6535,2325


##### 4. Составить табличку типа 'warehouse_name' , 'product', 'quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада).

In [845]:
percent_profit = orders_data1.groupby(['warehouse_name', 'product'], as_index=False)\
                .aggregate({'shipping_cost_for_each_product' : 'sum', 'total_cost_for_each_product' : 'sum'\
                                    , 'quantity' : 'sum'})

percent_profit['product_profit'] = \
                        (percent_profit['shipping_cost_for_each_product'] + \
                            percent_profit['total_cost_for_each_product']).astype(int)

percent_profit = percent_profit\
                .drop(['shipping_cost_for_each_product', 'total_cost_for_each_product'], axis=1)

percent_profit.head()

,warehouse_name,product,quantity,product_profit
0,Мордор,автограф Стаса Барецкого,4,2360
1,Мордор,билет в Израиль,9,8910
2,Мордор,зеленая пластинка,11,0
3,Мордор,ломтик июльского неба,3,1320
4,Мордор,плюмбус,6,1440


In [846]:
sum_cost = percent_profit.groupby('warehouse_name', as_index=False)\
            .aggregate({'product_profit' : 'sum'}).rename({'product_profit' : 'product_profit_sum'})
sum_cost = sum_cost.rename({'product_profit' : 'product_profit_sum'}, axis=1)
sum_cost.rename({'product_profit_sum' : 'total_profit_of_warehouse'}, axis=1)
sum_cost

,warehouse_name,product_profit_sum
0,Мордор,17050
1,гиперборея,38280
2,остров невезения,15610
3,отель Лето,27050
4,хутор близ Диканьки,37240


In [847]:
percent_profit = percent_profit.set_index('warehouse_name')\
                    .join(sum_cost.set_index('warehouse_name'), on='warehouse_name')
percent_profit.reset_index(inplace=True)
percent_profit.head()

,warehouse_name,product,quantity,product_profit,product_profit_sum
0,Мордор,автограф Стаса Барецкого,4,2360,17050
1,Мордор,билет в Израиль,9,8910,17050
2,Мордор,зеленая пластинка,11,0,17050
3,Мордор,ломтик июльского неба,3,1320,17050
4,Мордор,плюмбус,6,1440,17050


In [848]:
percent_profit['percent_profit_product_of_warehouse'] = 100 * percent_profit['product_profit']\
                             / percent_profit['product_profit_sum']
percent_profit = percent_profit.drop('product_profit_sum', axis=1)
percent_profit = percent_profit.rename({'product_profit' : 'profit'}, axis=1)
percent_profit.head(15)

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,4,2360,13.841642
1,Мордор,билет в Израиль,9,8910,52.258065
2,Мордор,зеленая пластинка,11,0,0.000000
3,Мордор,ломтик июльского неба,3,1320,7.741935
4,Мордор,плюмбус,6,1440,8.445748
5,Мордор,подписка на suppi-блог,8,1120,6.568915
6,Мордор,статуэтка Ленина,10,1900,11.143695
7,гиперборея,автограф Стаса Барецкого,12,6960,18.181818
8,гиперборея,билет в Израиль,21,20580,53.761755
9,гиперборея,зеленая пластинка,10,-100,-0.261233


##### 5. Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться 'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

In [850]:
percent_profit = percent_profit.sort_values(by=['percent_profit_product_of_warehouse'], ascending=False)
percent_profit.head()

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
21,отель Лето,билет в Израиль,15,14625,54.066543
8,гиперборея,билет в Израиль,21,20580,53.761755
1,Мордор,билет в Израиль,9,8910,52.258065
27,хутор близ Диканьки,автограф Стаса Барецкого,22,12870,34.559613
17,остров невезения,ломтик июльского неба,10,4450,28.507367


In [851]:
percent_profit["accumulated_percent_profit_product_of_warehouse"] = percent_profit\
                        .groupby('warehouse_name')['percent_profit_product_of_warehouse'].cumsum()

In [852]:
percent_profit = percent_profit.sort_values(['warehouse_name', 'percent_profit_product_of_warehouse'], ascending=False)
percent_profit

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
27,хутор близ Диканьки,автограф Стаса Барецкого,22,12870,34.559613,34.559613
28,хутор близ Диканьки,билет в Израиль,10,9850,26.450054,61.009667
31,хутор близ Диканьки,плюмбус,27,6345,17.038131,78.047798
33,хутор близ Диканьки,статуэтка Ленина,21,3885,10.432331,88.480129
30,хутор близ Диканьки,ломтик июльского неба,7,3045,8.176692,96.656821
32,хутор близ Диканьки,подписка на suppi-блог,10,1350,3.625134,100.281955
29,хутор близ Диканьки,зеленая пластинка,21,-105,-0.281955,100.000000
21,отель Лето,билет в Израиль,15,14625,54.066543,54.066543
20,отель Лето,автограф Стаса Барецкого,8,4600,17.005545,71.072089
23,отель Лето,ломтик июльского неба,9,3825,14.140481,85.212569


##### 6. Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A. Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

In [853]:
percent_profit['category'] = 'B'
percent_profit.loc[percent_profit['accumulated_percent_profit_product_of_warehouse']\
                               .astype(int) <= 70, 'category'] = 'A'
percent_profit.loc[percent_profit['accumulated_percent_profit_product_of_warehouse']\
                               .astype(int) > 90, 'category'] = 'C'
percent_profit

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
27,хутор близ Диканьки,автограф Стаса Барецкого,22,12870,34.559613,34.559613,A
28,хутор близ Диканьки,билет в Израиль,10,9850,26.450054,61.009667,A
31,хутор близ Диканьки,плюмбус,27,6345,17.038131,78.047798,B
33,хутор близ Диканьки,статуэтка Ленина,21,3885,10.432331,88.480129,B
30,хутор близ Диканьки,ломтик июльского неба,7,3045,8.176692,96.656821,C
32,хутор близ Диканьки,подписка на suppi-блог,10,1350,3.625134,100.281955,C
29,хутор близ Диканьки,зеленая пластинка,21,-105,-0.281955,100.000000,C
21,отель Лето,билет в Израиль,15,14625,54.066543,54.066543,A
20,отель Лето,автограф Стаса Барецкого,8,4600,17.005545,71.072089,B
23,отель Лето,ломтик июльского неба,9,3825,14.140481,85.212569,B
